In [114]:
from sklearn import preprocessing as pp
import numpy as np
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Lambda
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from numpy.random import multinomial as randm

# Current Counts = Current Counts + Note Length % 16 ?

def buildNotes():
    
    print("In buildNotes()")
    
    notes = []
    noteLengths = []
    
    for file in glob.glob("Music/*.mid"):
        
        try:
            midi = converter.parse(file)    
            
        except:
            print("MIDI file %s failed to parse" % file)
            continue
            
        print("Parsing %s" % file)
        notesToParse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notesToParse = s2.parts[0].recurse()
            print("Instrument Parts %d" % len(s2.parts))
            
        except: # file has notes in a flat structure
            notesToParse = midi.flat.notes
            
        # For each note in notesToParse (a stream of notes)
            
        for element in notesToParse:
            
            # Interesting parameters of notes that we may want to look at here.
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                noteLengths.append(str(float(element.quarterLength)))
                
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                noteLengths.append(str(float(element.quarterLength)))

    pickle.dump(notes, open('notes.p', 'wb'))
    pickle.dump(noteLengths, open('noteLengths.p', 'wb'))

    return notes, noteLengths

def prepareSeq(notes, noteLengths):

    seqLength = 4
    inputSize = len(notes) - seqLength
    categories = 2
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    # We only have 2 dimensions at the moment, pitch and length
    # noteIn - a collection of categorical arrays, each row is one category
    # noteOut - the integer which converts to the index of that category's respective
    # one hot encoding.  These categorical one hot encodings are later concatenated
    # columnwise to form a multi hot encoding.

    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]

    # Mapping of i:i+seqLength notes to note i + seqLength
    
    # X[0] sequence_in : [seqLength]
    # X[1] length_in : [seqLength]
    
    # Y[0] sequence_out : 1
    # Y[1] length_out : 1
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        
        length_in = noteLengths[i:i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteIn[1].append([lengthToInt[char] for char in length_in])
        
        noteOut[0].append(noteToInt[sequence_out])
        noteOut[1].append(lengthToInt[length_out])
        
    # npatterns = len(noteIn) (inputSize)

    # reshape the input into a format compatible with LSTM layers
    # noteIn = np.reshape(noteIn, (inputSize, seqLength, 1))
    
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
    
    print("noteIn shape:")
    print(noteIn.shape)
    print("noteOut shape:")
    print(noteOut.shape)
    print(noteIn)
    
    catNote = np_utils.to_categorical(noteOut[0])
    catLength = np_utils.to_categorical(noteOut[1])
    
    # Concatenate the input arrays row wise
    # Concatenate the categorical arrays along axis 1 (columnwise)
    
    print("noteIn:")
    print(noteIn)
    
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    dout = np.concatenate((catNote, catLength), 1)
    din = np.reshape(reshape, (inputSize, seqLength, categories))
    
    print("din shape:")
    print(din.shape)
    print("dout shape:")
    print(dout.shape)
    print("din:")
    print(din)
    
    print(len(catNote[0]))
    print(len(catLength[0]))
    
    print(din[0][0][0])
    print(din[0][0][1])

    return (din, dout)


In [115]:
# print(np.arange(10).reshape(5,2).T.flatten())
notes, noteLengths = buildNotes()
noteIn, noteOut = prepareSeq(notes, noteLengths)
print(noteIn.shape)
print(noteOut.shape)

In buildNotes()
Parsing Music\Fugue1.mid
noteIn shape:
(2, 736, 4)
noteOut shape:
(2, 736)
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.2        0.2        0.2        0.33333333]
  [0.2        0.2        0.33333333 0.        ]
  [0.2        0.33333333 0.         0.        ]
  ...
  [0.06666667 0.26666667 0.06666667 0.2       ]
  [0.26666667 0.06666667 0.2        0.06666667]
  [0.06666667 0.2        0.06666667 0.8       ]]]
noteIn:
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.2       

In [88]:
def createNet(noteIn, nvocab):

    print("In createNet()")
    
    print(noteIn.shape)
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    # Chris's code
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=False
    ))
    
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation(activation='relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    
    #model.add(Flatten())
    
    model.add(Dense(nvocab))
    model.add(Lambda(lambda x: x / 0.6))
    model.add(Activation(activation='softmax'))

    model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
    
    return model

def trainNet(_epochs=1):
    """ Train a Neural Network to generate music """
    notes, noteLengths = buildNotes()

    nvocab = len(set(notes)) + len(set(noteLengths))
    
    noteIn, noteOut = prepareSeq(notes, noteLengths)
    
    model = createNet(noteIn, nvocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(noteIn, noteOut, batch_size=64, epochs=_epochs, verbose=1, callbacks=callbacks_list)
    


In [89]:
trainNet(50)

In buildNotes()
Parsing Music\Fugue1.mid
noteIn shape:
(2, 736, 4)
noteOut shape:
(2, 736)
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333 0.13333333 0.13333333 0.2       ]
  [0.13333333 0.13333333 0.2        0.        ]
  [0.13333333 0.2        0.         0.        ]
  ...
  [0.06666667 0.8        0.06666667 0.13333333]
  [0.8        0.06666667 0.13333333 0.06666667]
  [0.06666667 0.13333333 0.06666667 0.66666667]]]
noteIn:
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333

In [118]:
def generate(model):
    
    print("In Generate")
    
    notes = pickle.load(open('notes.p', 'rb'))
    noteLengths = pickle.load(open('noteLengths.p', 'rb'))

    pitchSet = sorted(set(notes))
    lengthSet = sorted(set(noteLengths))
    
    # 1. Call to prepareSeqPred
    
    noteIn = prepareSeqPred(notes, noteLengths) 
    
    # print("Model Loaded")
    
    # 2. Call to genNotes
    
    predOut = genNotes(model, noteIn, pitchSet, lengthSet)
    
    # 3. Call to createMidi
    createMidi(predOut)

# 1.
    
def prepareSeqPred(notes, noteLengths):
    
    print("In Prepare Sequences Prediction")
    
    seqLength = 4
    inputSize = len(notes) - seqLength
    categories = 2
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        length_in = noteLengths[i:i + seqLength]
        
        sequence_out = notes[i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteOut[0].append(noteToInt[sequence_out])
        
        noteIn[1].append([lengthToInt[char] for char in length_in])
        noteOut[1].append(lengthToInt[length_out])
        
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
        
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    din = np.reshape(reshape, (inputSize, seqLength, categories))
        
    print("din shape:")
    print(din.shape)
    print("din:")
    print(din)

    return din

# 2.

def genNotes(model, noteIn, pitchSet, lengthSet):
    
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    
    inputSize = len(noteIn)
        
    start = np.random.randint(0, inputSize - 1)
    pattern = noteIn[start]
    
    print("pattern %s" % pattern)
        
    seqLength = len(pattern)
    categories = 2
    
    print("seqLength: %d" % seqLength)

    npitches = len(pitchSet)
    nlengths = len(lengthSet)
    
    print("npitches %d nlengths %d" % (npitches, nlengths))
    
    # TODO
    intToNote = dict((number, note) for number, note in enumerate(pitchSet))
    intToLength = dict((number, length) for number, length in enumerate(lengthSet))
    
    predOut = []
    
    print("In genNotes(): noteIn[start] = %s" % pattern)
    print("inputSize: %d" % inputSize)
    print("categories: %d" % categories)
    
    print("pitchSet %s" % pitchSet)
    print("lengthSet %s" % lengthSet)

    for i in range(100):
        
        prediction_input = np.reshape(pattern, (1, seqLength, categories))
        
        print("prediction_input:")
        print(prediction_input)
        
        ### Complete the line below
        prediction = model.predict(np.array(prediction_input))
        print(len(prediction[0]))
        
        print("prediction: %s" % prediction[0])
        print("prediction length: %d" % len(prediction[0]))
        
        predPitch = prediction[0][:npitches].astype('float64')
        predLength = prediction[0][npitches:].astype('float64')
        
        print("Before")
        print(predPitch)
        print(predLength)
        
        #predPitch /= sumNote
        #predLength /= sumLength
        
        print("sumNote %f sumLength %f" % (predPitch.sum(), predLength.sum()))
        
        predPitch /= predPitch.sum()
        predLength /= predLength.sum()
        
        print("After")
        print(predPitch)
        print(predLength)
        
        print("sumNote %f sumLength %f" % (predPitch.sum(), predLength.sum()))
        
        array = randm(1, predPitch)
        print("array predPitch: %s" % array)
        indexPitch = np.argmax(array)
        
        array = randm(1, predLength)
        print("array predLength: %s" % array)
        indexLength = np.argmax(array)
        
        #TODO return tuple pitch, length
        result = np.array([intToNote[indexPitch], intToLength[indexLength]])
        predOut.append(result)
        
        print("result %s" % result)
        
        append = np.reshape(np.array([indexPitch/float(npitches), indexLength/float(nlengths)]), (1,2))
        pattern = np.concatenate((pattern, append))
        
        print("pattern: %s" % pattern)
        
        pattern = pattern[1:len(pattern)]
        
    print(predOut)

    return predOut

# 3.
    
def createMidi(predOut):
    print("In createMidi()")
    offset = 0
    output_notes = []
    for pattern in predOut:
        if ('.' in pattern[0]) or pattern[0].isdigit():
            notes_in_chord = pattern[0].split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                if pattern[1] != '0.0':
                    new_note.quarterLength = float(pattern[1]) # Assign duration
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern[0])
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            if pattern[1] != '0.0':
                new_note.quarterLength = float(pattern[1]) # Assign duration
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')


In [90]:
model = 0

In [119]:
# Load a previous model (dependency)
if model == 0:
    model = load_model("weights2-improvement-50-5.9896-bigger.hdf5")
generate(model)

In Generate
In Prepare Sequences Prediction
din shape:
(736, 4, 2)
din:
[[[0.46511628 0.2       ]
  [0.55813953 0.2       ]
  [0.65116279 0.2       ]
  [0.81395349 0.33333333]]

 [[0.55813953 0.2       ]
  [0.65116279 0.2       ]
  [0.81395349 0.33333333]
  [0.95348837 0.        ]]

 [[0.65116279 0.2       ]
  [0.81395349 0.33333333]
  [0.95348837 0.        ]
  [0.81395349 0.        ]]

 ...

 [[0.97674419 0.06666667]
  [0.1627907  0.26666667]
  [0.8372093  0.06666667]
  [0.58139535 0.2       ]]

 [[0.1627907  0.26666667]
  [0.8372093  0.06666667]
  [0.58139535 0.2       ]
  [0.34883721 0.06666667]]

 [[0.8372093  0.06666667]
  [0.58139535 0.2       ]
  [0.34883721 0.06666667]
  [0.51162791 0.8       ]]]
pattern [[0.02325581 0.        ]
 [0.30232558 0.2       ]
 [0.93023256 0.2       ]
 [0.02325581 0.        ]]
seqLength: 4
npitches 43 nlengths 15
In genNotes(): noteIn[start] = [[0.02325581 0.        ]
 [0.30232558 0.2       ]
 [0.93023256 0.2       ]
 [0.02325581 0.        ]]
inputSiz

array predPitch: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0]
array predLength: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
result ['G3' '0.25']
pattern: [[0.23255814 0.        ]
 [0.46511628 0.06666667]
 [0.6744186  0.13333333]
 [0.1627907  0.06666667]
 [0.93023256 0.06666667]]
prediction_input:
[[[0.46511628 0.06666667]
  [0.6744186  0.13333333]
  [0.1627907  0.06666667]
  [0.93023256 0.06666667]]]
58
prediction: [1.61507295e-03 1.64202263e-03 8.03354778e-05 1.79491908e-04
 6.32419833e-04 4.19778284e-03 3.63606550e-02 2.58448645e-02
 1.40847202e-04 3.22952663e-04 1.23388227e-03 3.27724774e-06
 9.18538950e-04 9.27003857e-04 2.75398418e-03 1.41046941e-04
 1.79218754e-04 1.11868989e-03 4.06485284e-04 7.48233986e-04
 1.66813303e-02 9.03000910e-05 8.56514162e-05 3.88473459e-03
 3.38973803e-03 4.33783652e-03 1.75861751e-05 1.27821509e-02
 6.61270991e-02 8.85575917e-03 3.14921228e-04 1.62347760e-02
 6.25884277e-04 3.24036460e-04 6.15693927e-02 7.46694976e-04


58
prediction: [1.5472643e-03 1.7764436e-03 4.7975318e-05 2.9968683e-04 9.0796000e-04
 6.1041117e-03 4.2404719e-02 8.1210218e-02 1.1544582e-04 2.6054680e-04
 1.0172452e-03 1.3244464e-06 2.8317038e-04 9.6699042e-04 1.5712621e-03
 7.0238661e-05 1.4887715e-04 1.0971642e-03 4.3736902e-04 1.9452994e-04
 1.6471675e-02 4.8665010e-05 5.9978815e-05 2.6873245e-03 2.8622972e-03
 2.1972379e-03 1.2544015e-05 1.9471144e-02 1.3713349e-01 5.7558334e-03
 1.7154108e-04 9.8319398e-03 2.0391736e-04 2.1413351e-04 6.4113580e-02
 4.4703225e-04 1.0543263e-03 1.6036804e-03 4.9643018e-05 1.6904151e-04
 2.3740041e-03 3.0803264e-03 3.0843401e-03 1.5930681e-02 5.0680596e-01
 1.3279414e-02 2.8695210e-03 1.6184092e-02 3.1323769e-05 3.6644844e-05
 5.6258141e-04 2.4804936e-03 6.6641855e-06 1.8932751e-05 8.2238833e-04
 1.9699434e-02 5.4076877e-03 2.3038762e-03]
prediction length: 58
Before
[1.54726428e-03 1.77644356e-03 4.79753180e-05 2.99686828e-04
 9.07959999e-04 6.10411167e-03 4.24047187e-02 8.12102184e-02
 1.154458

array predPitch: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0]
array predLength: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
result ['F5' '0.25']
pattern: [[0.11627907 0.06666667]
 [0.53488372 0.26666667]
 [0.62790698 0.06666667]
 [0.72093023 0.06666667]
 [0.8372093  0.06666667]]
prediction_input:
[[[0.53488372 0.26666667]
  [0.62790698 0.06666667]
  [0.72093023 0.06666667]
  [0.8372093  0.06666667]]]
58
prediction: [1.7327443e-03 3.5109845e-04 9.9871522e-06 8.8804831e-05 1.8440833e-04
 1.2821411e-02 7.0657112e-02 4.1946743e-02 7.6009586e-05 1.6445840e-04
 4.2991215e-04 6.2238223e-07 1.6830047e-04 1.4238650e-03 1.4154215e-03
 5.5095017e-05 3.3109212e-05 9.4897574e-04 1.1629029e-04 3.3338560e-04
 1.2860917e-02 6.9279100e-05 1.1730999e-05 1.0902336e-02 6.0678697e-03
 1.1397309e-03 7.3784327e-06 6.0621086e-03 1.7037825e-01 8.0589810e-03
 3.0966385e-04 8.1661111e-03 1.4206029e-04 2.5736654e-04 8.0898859e-02
 1.2003497e-03 1.2858955e-03 3.2517554e-03 4.2889962

58
prediction: [1.15336373e-03 5.92112425e-04 6.05023160e-06 4.94473032e-04
 3.92601651e-04 4.46465276e-02 8.87408927e-02 8.60715881e-02
 7.24225320e-05 1.18437172e-04 1.42781180e-04 3.06780294e-07
 4.88323712e-05 1.05935999e-03 2.05382472e-04 1.99863298e-05
 3.74799129e-05 1.46907242e-03 8.79271829e-05 6.40467697e-05
 1.00865429e-02 1.50248479e-05 6.42140685e-06 8.39294493e-03
 4.23294865e-03 2.97919876e-04 1.20263448e-05 5.81393903e-03
 2.79045790e-01 3.64302495e-03 1.03535596e-04 3.29425395e-03
 1.79911713e-05 7.80986738e-05 1.32985845e-01 4.58288792e-04
 5.58394939e-04 3.33510782e-03 9.85827319e-06 3.90441244e-04
 4.03727085e-04 1.22501072e-03 4.42529796e-04 1.85811240e-02
 2.44232208e-01 7.52826035e-03 2.15484621e-03 5.30677848e-03
 1.11276968e-05 2.23317511e-05 5.34851511e-04 1.89040583e-02
 1.11881263e-05 5.41372901e-06 4.45232290e-04 1.31861994e-03
 2.00346503e-02 6.36167242e-04]
prediction length: 58
Before
[1.15336373e-03 5.92112425e-04 6.05023160e-06 4.94473032e-04
 3.926016

In [ ]:
generate(model)